<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ngram_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Language Modeling**

## **Goal:** compute a probabilty distribution over all possible sentences:


## $$p(W) = p(w_1, w_2, ..., w_T)$$

## This unsupervised learning problem can be framed as a sequence of supervised learning problems:

## $$p(W) = p(w_1) * p(w_2|w_1) * ... * p(w_T|w_1, ..., w_{T-1})$$

## If we have N sentences, each of them with T words / tokens, then we want to max:

## $$log p(W) = \sum_{n = 1}^N \sum_{i=1}^{T} log p(w_i | w_{<i})$$




# **N-gram language model**

## **Goal:** estimate the n-gram probabilities using counts of sequences of n consecutive words

## Given a sequence of words $w$, we want to compute

##  $$P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1})$$

## Where $w_i$ is the i-th word of the sequence.

## $$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) = \frac{p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$$

## **Key Idea:** We can estimate the probabilities using counts of n-grams in our dataset 


# Let's see this in Practice

### Install if needed

TODO: should we install as needed and import as needed or all at once?

In [1]:
# # run if you dont have it installed
# !pip install more_itertools
# !pip install spacy# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager\
# !python -m spacy download en_core_web_sm

### Imports

In [2]:
import spacy
import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy
import itertools
from operator import itemgetter 
from glob import glob
from tqdm import tqdm_notebook, tqdm
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import string
import re
import more_itertools as mit  # not built-in package
import torch
import torchtext
import torchtext.data as data
from torchtext import vocab
from collections import Counter
import re
from torchtext.data import TabularDataset 

In [3]:
torch.manual_seed(1)


## Data Processing

In [4]:
class AmazonReviewsDataset(TabularDataset):
    
    urls = [
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz',
           'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Tools_and_Home_Improvement_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Automotive_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz',
#             'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz',
        ]
    name='amazonreviews'
    dirname='processed'

In [5]:
# download_done = AmazonReviewsDataset.download(root='data/', check=True)

In [6]:
# RETOK = re.compile(r'\w+|[^\w\s]|\n', re.UNICODE)

# def tokenize(s):
#     return RETOK.findall(s)

# text_field = data.Field(sequential=True, tokenize=tokenize, include_lengths=True, use_vocab=True, lower=True, batch_first=True)

In [7]:
# dataset = AmazonReviewsDataset(path='/home/roberta/ammi-2019-nlp/01-day-LM/data/amazonreviews/reviews_Sports_and_Outdoors_5.json', format='json', fields={'reviewText': ('reviewText', text_field), 'summary': ('summary', text_field)})

In [8]:
# # lets check it
# # lets use fstrings btw
# print(f'Number of samples : {len(dataset.examples)}')

# for ex in dataset.examples:
#     print(f'Review: \n {ex.reviewText} \n\n Summary: \n {ex.summary}')
#     break

In [9]:
# # convert the dataset to a list of strings
# # each string represents a review
# all_reviews = []
# for ex in dataset.examples:
#     all_reviews.append(ex.reviewText)
# len(all_reviews)


In [10]:
# _tqdm = tqdm_notebook  # prolly you need jupyter widget for this, change for tqdm for simple tqdm

# NUM_SENTENCES = len(all_reviews)
# NUM_SENTENCES_TRAIN = int(0.8*NUM_SENTENCES)
# NUM_SENTENCES_TEST = int(0.1*NUM_SENTENCES)
# NUM_SENTENCES_VALID = NUM_SENTENCES - NUM_SENTENCES_TRAIN - NUM_SENTENCES_TEST

# train_reviews = all_reviews[:NUM_SENTENCES_TRAIN]
# test_reviews = all_reviews[NUM_SENTENCES_TRAIN:NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST]
# valid_reviews = all_reviews[NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST:NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST+NUM_SENTENCES_VALID]

# train_reviews = [str(t) for t in train_reviews]
# test_reviews = [str(t) for t in test_reviews]
# valid_reviews = [str(t) for t in valid_reviews]


#### TODO make a function to convert list of urls to txt files with train, test, and valid data

In [11]:
# with open('../data/amazon_reviews_sports_train.txt', 'w') as f:
#     for item in train_reviews:
#         f.write("%s\n" % item)

 

In [12]:
# with open('../data/amazon_reviews_sports_valid.txt', 'w') as f:
#     for item in valid_reviews:
#         f.write("%s\n" % item)
        


In [13]:
# with open('../data/amazon_reviews_sports_test.txt', 'w') as f:
#     for item in test_reviews:
#         f.write("%s\n" % item)

### Load Data From .txt Files

In [21]:
# Train Data
train_data = open('../data/amazon_reviews_sports_train.txt').readlines()

In [22]:
# Inspect Train Data        
len(train_data), type(train_data), train_data[0], type(train_data[0]), len(train_data[0])

(237069,
 list,
 "['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy']\n",
 str,
 227)

In [23]:
# Validation Data
valid_data = open('../data/amazon_reviews_sports_valid.txt').readlines()

In [24]:
# Inspect Valid Data
len(valid_data), type(valid_data), valid_data[0], type(valid_data[0]), len(valid_data[0])

(29635,
 list,
 "['great', 'sling', 'bag', '.', 'i', 'used', 'it', 'for', 'a', 'trip', 'to', 'disney', 'and', 'this', 'bag', 'worked', 'very', 'well', '.', 'it', 'has', 'enough', 'pockets', 'to', 'carry', 'all', 'the', 'essentials', 'needed', 'for', 'a', 'vacation', 'to', 'disney', 'in', 'june', '.', 'i', 'was', 'able', 'to', 'carry', 'so', 'mush', 'stuff', 'in', 'this', 'bag', 'my', 'wife', 'did', 'not', 'have', 'to', 'carry', 'a', 'back', 'pack', '.', 'i', 'gonna', 'to', 'carry', 'this', 'bag', 'for', 'all', 'my', 'trips', 'from', 'now', 'on', 'and', 'it', 'worked', 'well', 'through', 'all', 'the', 'airport', 'security', 'checks', '.', 'i', 'highly', 'recommend', 'this', 'bag', '.']\n",
 str,
 677)

In [25]:
# Test Data
test_data = open('../data/amazon_reviews_sports_test.txt').readlines()

In [26]:
# Inspect Test Data
len(test_data), type(test_data), test_data[0], type(test_data[0]), len(test_data[0])

(29633,
 list,
 '[\'this\', \'is\', \'the\', \'most\', \'affordable\', \'/\', \'highest\', \'quality\', \'made\', \'in\', \'the\', \'usa\', \'knife\', \'i\', \'have\', \'found\', \'.\', \'the\', \'knife\', \'is\', \'very\', \'lightweight\', \',\', \'lighter\', \'than\', \'my\', \'kershaw\', \'blur\', \'and\', \'its\', \'a\', \'bit\', \'thinner\', \'too\', \'.\', \'typically\', \'really\', \'lightweight\', \'knives\', \'have\', \'frn\', \'handles\', \'like\', \'the\', \'endura\', \'4\', \'or\', \'the\', \'griptillian\', \'and\', \'that\', "\'", \'s\', \'fine\', \'but\', \'that\', \'stuff\', \'does\', \'flex\', \'if\', \'u\', \'squeeze\', \'hard\', \'enough\', \'.\', \'the\', \'aluminum\', \'handles\', \'in\', \'the\', \'knockout\', \'feel\', \'really\', \'strong\', \'and\', \'have\', \'no\', \'give\', \'to\', \'them\', \'and\', \'i\', \'prefer\', \'the\', \'feel\', \'of\', \'metal\', \'over\', \'plastic\', \'any\', \'day\', \'.\', \'there\', \'was\', \'a\', \'little\', \'rattle\', \'if\

### Process the Data

In [27]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')               
punctuations = string.punctuation   
TAG_RE = re.compile(r'<[^>]+>') # get rid off HTML tags from the data

def remove_tags(text):
    return TAG_RE.sub('', text)

def lower_case_remove_punc(parsed):
    return [token.text.lower() for token in parsed if (token.text not in punctuations)] #and (token.is_stop is False)]

def tokenize_dataset(dataset):
   # tokenize each sentence -- each tokenized sentence will be an element in token_dataset
    token_dataset = []
    # tokenize all words -- each token will be an item in all_tokens (in the order given by the list of sentences)
    all_tokens = []     # all the tokens -- 

    for sample in tqdm(tokenizer.pipe(dataset, disable=['parser', 'tagger', 'ner'], batch_size=512, n_threads=1)):
        tokens = lower_case_remove_punc(sample)
        token_dataset.append(tokens)
        all_tokens += tokens
        
    return token_dataset, all_tokens

In [28]:
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [29]:
TAG_RE

re.compile(r'<[^>]+>', re.UNICODE)

In [30]:
train_data[:2]

["['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy']\n",
 '[\'i\', \'had\', \'a\', \'factory\', \'glock\', \'tool\', \'that\', \'i\', \'was\', \'using\', \'for\', \'my\', \'glock\', \'26\', \',\', \'27\', \',\', \'and\', \'17\', \'.\', \'i\', "\'", \'ve\', \'since\', \'lost\', \'it\', \'and\', \'had\', \'needed\', \'another\', \'.\', \'since\', \'i\', "\'", \'ve\', \'used\', \'ghost\', \'products\', \'prior\', \',\', \'and\', \'know\', \'that\', \'they\', \'are\', \'reliable\', \',\', \'i\', \'had\', \'decided\', \'to\', \'order\', \'this\', \'one\', \'.\', \'sure\', \'enough\', \',\', \'this\', \'is\', \'just\', \'as\', \'good\', \'as\', \'a\', \'factory\', \'tool\', \'.\']\n']

In [31]:
# TODO: for now only work with small subset of the data -- switch to all data later
train_data = train_data[:800]
test_data = test_data[:100]
valid_data = valid_data[:100]

In [32]:
train_data[0], len(train_data)

("['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy']\n",
 800)

In [33]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = tokenize_dataset(train_data)
test_data_tokenized, all_tokens_test = tokenize_dataset(test_data)
valid_data_tokenized, all_tokens_valid = tokenize_dataset(valid_data)


800it [00:04, 175.49it/s]
100it [00:00, 224.68it/s]
100it [00:00, 226.14it/s]


Let's look at the tokenized data!

In [34]:
# Number of Tokenized Sentences == Number of All Sentences in the Dataset
# Compare the Tokenized Sentences with the Original Ones
len(train_data_tokenized), train_data_tokenized[:2], train_data[:2]

(800,
 [['this',
   'came',
   'in',
   'on',
   'time',
   'and',
   'i',
   'am',
   'veru',
   'happy',
   'with',
   'it',
   'i',
   'haved',
   'used',
   'it',
   'already',
   'and',
   'it',
   'makes',
   'taking',
   'out',
   'the',
   'pins',
   'in',
   'my',
   'glock',
   '32',
   'very',
   'easy',
   '\n'],
  ['i',
   'had',
   'a',
   'factory',
   'glock',
   'tool',
   'that',
   'i',
   'was',
   'using',
   'for',
   'my',
   'glock',
   '26',
   '27',
   'and',
   '17',
   'i',
   've',
   'since',
   'lost',
   'it',
   'and',
   'had',
   'needed',
   'another',
   'since',
   'i',
   've',
   'used',
   'ghost',
   'products',
   'prior',
   'and',
   'know',
   'that',
   'they',
   'are',
   'reliable',
   'i',
   'had',
   'decided',
   'to',
   'order',
   'this',
   'one',
   'sure',
   'enough',
   'this',
   'is',
   'just',
   'as',
   'good',
   'as',
   'a',
   'factory',
   'tool',
   '\n']],
 ["['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am'

In [35]:
# Number of All Tokens
len(all_tokens_train), all_tokens_train[:2]

(78453, ['this', 'came'])

#### Build the Vocabulary 


In [36]:
# TODO: do we use both train and valid and not test for this??
voc = list(set(all_tokens_train + all_tokens_valid))
print('Word vocabulary size: {} words'.format(len(voc)))             

Word vocabulary size: 6281 words


### CORPUS ANALYSIS (Train + Valid Data)

#### Number of Tokens in the Corpus Data


In [37]:
print("Number of All Tokens ", len(all_tokens_train) + len(all_tokens_valid))

Number of All Tokens  85962


In [38]:
print("Number of All UNIQUE Tokens ", len(voc))

Number of All UNIQUE Tokens  6281


#### Number of Sentences in the Train Data


In [39]:
print("Number of Sentences ", len(train_data_tokenized), len(valid_data_tokenized))

Number of Sentences  800 100


#### Count how often each sentence length occurs. Visualize this! 

#### Average sentence length

## N-grams

#### Generate a list of words and their corresponding frequencies. Which are the 10 most frequent words?



### Function for padding the sentences with special markers sentence beginning and end, i.e. $<bos>$ and $<eos>$

In [40]:
def pad_sentences(input_list, n):
    result_list = []
    for l in input_list:
        padded = ["<bos>" for i in range((n - 1))] + l +["<eos>" for i in range((n - 1))]
        result_list.append(padded)
    return result_list

In [41]:
NGRAM = 2  # change this to make different N grams for each token

train_data_padded = pad_sentences(train_data_tokenized, NGRAM)
test_data_padded = pad_sentences(test_data_tokenized, NGRAM)
valid_data_padded = pad_sentences(valid_data_tokenized, NGRAM)


Let's check our padding!

In [42]:
train_data_padded[0]

['<bos>',
 'this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'i',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 'i',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 '32',
 'very',
 'easy',
 '\n',
 '<eos>']

### Function for finding all N-grams

In [43]:
def find_ngrams(input_list, n):
    result_list = []
    for l in input_list:
        result_list.append(list(zip(*[l[i:] for i in range(n)])))
    return result_list

#### Convert the dataset to its corresponding n-gram version

In [44]:
NGRAM = 2  # change this to make different N grams for each token

# now make train and valid dicts
train_data_ngram = find_ngrams(train_data_padded, NGRAM)
valid_data_ngram = find_ngrams(valid_data_padded, NGRAM)
test_data_ngram = find_ngrams(test_data_padded, NGRAM)

Let's check our n-grams!

In [45]:
train_data_ngram[0]

[('<bos>', 'this'),
 ('this', 'came'),
 ('came', 'in'),
 ('in', 'on'),
 ('on', 'time'),
 ('time', 'and'),
 ('and', 'i'),
 ('i', 'am'),
 ('am', 'veru'),
 ('veru', 'happy'),
 ('happy', 'with'),
 ('with', 'it'),
 ('it', 'i'),
 ('i', 'haved'),
 ('haved', 'used'),
 ('used', 'it'),
 ('it', 'already'),
 ('already', 'and'),
 ('and', 'it'),
 ('it', 'makes'),
 ('makes', 'taking'),
 ('taking', 'out'),
 ('out', 'the'),
 ('the', 'pins'),
 ('pins', 'in'),
 ('in', 'my'),
 ('my', 'glock'),
 ('glock', '32'),
 ('32', 'very'),
 ('very', 'easy'),
 ('easy', '\n'),
 ('\n', '<eos>')]

#### Generate a complete list of trigrams occurring in the corpus. Which are the 10 most frequent trigrams?



#### Determine count statistics on the trigram frequencies, i.e. compute so-called count-counts (how many trigrams occur once, twice, : : :). Plot their distribution.

### Create N-gram Vocabulary with Corresponding Counts

In [46]:
max_vocab_size = 100000

all_train_tokens = list(mit.flatten(train_data_ngram + valid_data_ngram))
counted_tokens = Counter(all_train_tokens)

vocab, count = zip(*counted_tokens.most_common(max_vocab_size))


In [47]:
# Let's look at some numbers!
len(all_train_tokens), len(vocab), len(count)

(86862, 41952, 41952)

In [48]:
vocab[:2], count[:2]

((('\n', '<eos>'), ('of', 'the')), (900, 332))

### Create N-gram Dictionary

In [49]:
# save index 1 for unk, 0 for pad, 1 for bos, 2 for eos
PAD_IDX = 0
UNK_IDX = 1
BOS_IDX = 2
EOS_IDX = 3

id2token = list(vocab)
token2id = dict(zip(vocab, range(4, 4+len(vocab)))) 
id2token = ['<pad>', '<unk>', '<bos>', '<eos>'] + id2token

token2id['<pad>'] = PAD_IDX 
token2id['<unk>'] = UNK_IDX
token2id['<bos>'] = BOS_IDX 
token2id['<eos>'] = EOS_IDX

In [50]:
len(id2token), len(token2id)

(41956, 41956)

In [51]:
id2token[:10], token2id['<pad>'],  token2id['of', 'the']

(['<pad>',
  '<unk>',
  '<bos>',
  '<eos>',
  ('\n', '<eos>'),
  ('of', 'the'),
  ('<bos>', 'i'),
  ('it', 's'),
  ('i', 'have'),
  ('it', 'is')],
 0,
 5)

Lets check the dictionary by loading random token from it!


In [52]:
random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 8830 ; token ('splinter', 'removal')
Token ('splinter', 'removal'); token id 8830


#### Generate a list of trigrams in the corpus using only the words in the vocabulary. 

#### Generate count statistics of the trigram frequencies for this modified corpus as well. What do you notice in comparison to the previous exercise?

#### Determine the out-of-vocabulary (OOV) rate, i.e. the percentage of running words in the corpus which are not covered by the vocabulary.

### Functions for Converting from Token to ID and back

In [53]:
def _text2id(doc):
    return [token2id[t] if t in token2id else UNK_IDX for t in doc]

def _id2text(vec):
    return [id2token[i] for i in vec]
    

In [54]:
train_data_id = []
for d in train_data_ngram:
    train_data_id.append(_text2id(d))
    
valid_data_id = []
for d in valid_data_ngram:
    valid_data_id.append(_text2id(d))
    
train_data_id_merged = []
for d in train_data_id:
    train_data_id_merged.append((d, 0))

valid_data_id_merged = []
for d in valid_data_id:
    valid_data_id_merged.append((d, 0))


In [55]:
# let's look at what we created
train_data_id[0][:2], train_data_ngram[0][:2], train_data_id_merged[0], len(train_data_id_merged), len(train_data_id)

([24, 9866],
 [('<bos>', 'this'), ('this', 'came')],
 ([24,
   9866,
   2508,
   3418,
   3419,
   370,
   19,
   52,
   9867,
   9868,
   399,
   136,
   139,
   9869,
   9870,
   435,
   5240,
   9871,
   21,
   887,
   9872,
   9873,
   324,
   9874,
   9875,
   31,
   1316,
   9876,
   9877,
   702,
   9878,
   4],
  0),
 800,
 800)

In Part 1 you generated a trigram frequency list for a given vocabulary.
Determine the list of bigram frequencies from it by summing over the first word position:
N(v;w) = N(; v;w) =
X
u
N(u; v;w)
Analogously, recompute the frequencies of unigrams.
Now, extract bigrams/unigrams directly from the corpus using your implementation from part 1
and compare it to the recomputed versions. What do you notice? How could you x this problem
without changing the recomputation method?

### Function for Getting N-gram counts for already tokenized data

In [141]:
def ngram_counts(data, n):
    print("data item ", data[0], "\n")
    data_pad = pad_sentences(data, n)
    print("padded item ", data_pad[0], "\n")
    ngram_data = find_ngrams(data_pad, n)
    print("ngram item ", ngram_data[0], "\n")
    all_train_tokens = list(mit.flatten(ngram_data))
    counted_tokens = Counter(all_train_tokens)
    vocab, count = zip(*counted_tokens.most_common(max_vocab_size))
    print("vocab item ", vocab[0], "\n")
    print("count item ", count[0], "\n")
    return vocab, count

### Unigram Counts

In [142]:
train_data_tokenized[0]

['this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'i',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 'i',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 '32',
 'very',
 'easy',
 '\n']

In [143]:
vocab_unigrams, count_unigrams = ngram_counts(train_data_tokenized, 1)
vocab_unigrams[:2], count_unigrams[:2]

data item  ['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy', '\n'] 

padded item  ['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy', '\n'] 

ngram item  [('this',), ('came',), ('in',), ('on',), ('time',), ('and',), ('i',), ('am',), ('veru',), ('happy',), ('with',), ('it',), ('i',), ('haved',), ('used',), ('it',), ('already',), ('and',), ('it',), ('makes',), ('taking',), ('out',), ('the',), ('pins',), ('in',), ('my',), ('glock',), ('32',), ('very',), ('easy',), ('\n',)] 

vocab item  ('the',) 

count item  3752 



((('the',), ('i',)), (3752, 2508))

### Bigram Counts

In [144]:
vocab_bigrams, count_bigrams = ngram_counts(train_data_tokenized, 2)
vocab_bigrams[:2], count_bigrams[:2]

data item  ['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy', '\n'] 

padded item  ['<bos>', 'this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy', '\n', '<eos>'] 

ngram item  [('<bos>', 'this'), ('this', 'came'), ('came', 'in'), ('in', 'on'), ('on', 'time'), ('time', 'and'), ('and', 'i'), ('i', 'am'), ('am', 'veru'), ('veru', 'happy'), ('happy', 'with'), ('with', 'it'), ('it', 'i'), ('i', 'haved'), ('haved', 'used'), ('used', 'it'), ('it', 'already'), ('already', 'and'), ('and', 'it'), ('it', 'makes'), ('makes', 'taking'), ('taking', 'out'), ('out', 'the'), ('the', 'pins'), ('pins', 'in'), ('in', 'my'), ('my', 'glock'), ('glock', '32'), ('32', 'very'), 

((('\n', '<eos>'), ('of', 'the')), (800, 299))

### Trigram Counts

In [145]:
vocab_trigrams, count_trigrams = ngram_counts(train_data_tokenized, 3)
vocab_trigrams[:2], count_trigrams[:2]

data item  ['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy', '\n'] 

padded item  ['<bos>', '<bos>', 'this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy', '\n', '<eos>', '<eos>'] 

ngram item  [('<bos>', '<bos>', 'this'), ('<bos>', 'this', 'came'), ('this', 'came', 'in'), ('came', 'in', 'on'), ('in', 'on', 'time'), ('on', 'time', 'and'), ('time', 'and', 'i'), ('and', 'i', 'am'), ('i', 'am', 'veru'), ('am', 'veru', 'happy'), ('veru', 'happy', 'with'), ('happy', 'with', 'it'), ('with', 'it', 'i'), ('it', 'i', 'haved'), ('i', 'haved', 'used'), ('haved', 'used', 'it'), ('used', 'it', 'already'), ('it', 'already', 'and'), ('already', 'and', 'it'), ('and', 'i

((('\n', '<eos>', '<eos>'), ('<bos>', '<bos>', 'i')), (800, 277))

#### Determine the list of bigram frequencies from it by summing over the first word position:

#### $ N(u, v, w) = \sum_u N(\cdot, v, w) $

#### Do the same for the unigrams. Do we get the same numbers as before?

## N-gram Probabilities

## $$P(w|w_{−n}, ..., w_{−2}, w_{−1}) \approx \frac{c(w_{−n}, ..., w_{−2}, w_{−1}, w)}{\sum_{w \in V} c(w_{−n}, ..., w_{−2}, w_{−1}, w)}$$


## Smoothing -- absolute discounting with interpolation?

## Bigram Probabilities

### $$p_{bi} = max \large{ \frac{N(v, w) - b_{bi}}{N(v), 0}  + b_{bi} \frac{V - N_0(v, \cdot)}{N(v)} p_{uni(w)} \large}$$

### $$p_{uni} = max \large{ \frac{N(w) - b_{uni}}{N, 0}  + b_{uni} \frac{V - N_0(\cdot)}{N} \frac{1}{V}}$$

### $$b_{bi} = \frac{N_1(\cdot, \cdot)}{N_1(\cdot, \cdot) + 2*N_2(\cdot, \cdot)}$$

### $$b_{uni} = \frac{N_1(\cdot)}{N_1(\cdot) + 2*N_2(\cdot)}$$


### $$N_r(\cdot) = \sum_{w: N(w) = r} 1$$

### $$N_r(\cdot, \cdot) = \sum_{v, w: N(v, w) = r} 1$$

### $$N_r(v, \cdot) = \sum_{w: N(v, w) = r} 1$$

### V is the number of words in the vocabulary

### $N_r(\cdot, \cdot)$ and $N_r(\cdot)$  are the count-counts for bigrams and unigrams respectively $


In [ ]:
# TODO: Let's compute the bigram frequencies / probabilities



### Let's check that the probabilities sum up to one
### $$\sum_w p_{bi}(w|v) = \sum_w p_{uni}(w) = 1$$



In [ ]:
# TODO: compute the sums



#### Let's look at some examples and see if they make sense

### Perplexity

### $PP = exp(-\frac{LL}{\sum_k(N_k + 1)})$

### $LL = \sum_{k=1}^{K} \sum_{n=1}^{N_k + 1} log p_{bi}(w_{k,n} | w_{k,n-1})$

### Sentece Probability

### Sentence Generation

## Making PyTorch Dataset out of our set of dicts

In [148]:
class ImdbDataset(Dataset):
    def __init__(self, data_list, max_inp_length=None, device='cpu'):
        """
        data_list is a list of tuples: (x,y) where x is a list of ids and y is a label
        """
        self.data = data_list
        self.max_len = max_inp_length
        self.data_tensors = []
        for (i, t) in tqdm_notebook(self.data):
            self.data_tensors.append((torch.LongTensor(i[:self.max_len]), torch.LongTensor([t])))
            #TODO: fix error regarding to(device)
#             self.data_tensors.append((torch.LongTensor(i[:self.max_len]).to(device), torch.LongTensor([t]).to(device)))
              
    def __getitem__(self, key):
        (inp, tgt) = self.data_tensors[key]
        
        return inp, tgt, len(inp)

    def __len__(self):
        return len(self.data)

def pad(tensor, length, dim=0, pad=0):
    """Pad tensor to a specific length.
    :param tensor: vector to pad
    :param length: new length
    :param dim: (default 0) dimension to pad
    :returns: padded tensor if the tensor is shorter than length
    """
    if tensor.size(dim) < length:
        return torch.cat(
            [tensor, tensor.new(*tensor.size()[:dim],
                                length - tensor.size(dim),
                                *tensor.size()[dim + 1:]).fill_(pad)],
            dim=dim)
    else:
        return tensor
    
def batchify(batch):
    maxlen = max(batch, key = itemgetter(2))[-1]
    batch_list = []
    target_list = []
    for b in batch:
        batch_list.append(pad(b[0], maxlen, dim=0, pad=PAD_IDX))
        target_list.append(b[1])
    input_batch = torch.stack(batch_list, 0)
    target_batch = torch.stack(target_list, 0)
    
    return input_batch, target_batch

In [149]:
train_dataset = ImdbDataset(train_data_id_merged, max_inp_length=None, device='cuda')
train_loader = DataLoader(train_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

valid_dataset = ImdbDataset(valid_data_id_merged, max_inp_length=None, device='cuda')
valid_loader = DataLoader(valid_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

In [16]:
out = vocab.FastText(language='en')

.vector_cache/wiki.en.vec:  19%|█▉        | 1.25G/6.60G [11:49<50:35, 1.76MB/s]


KeyboardInterrupt: 

In [ ]:
text_field.build_vocab(dataset, max_size=30000, vectors=out)

In [ ]:
# making a batch iterator
train_loader = data.BucketIterator(dataset=dataset, batch_size=4, sort_key=lambda x: len(x.reviewText), device=torch.device('cpu'), sort_within_batch=True, repeat=False)

In [ ]:
batch = next(iter(train_loader))
print(batch)

In [ ]:
def _vec2txt(vec):
    return [text_field.vocab.itos[t] for t in vec]

In [ ]:
print(batch.reviewText[0][0])
print(_vec2txt(batch.reviewText[0][0]))

# **Language Modeling**

**Goal:** compute a probabilty distribution over all possible sentences:


$p(W) = p(w_1, w_2, ..., w_T)$ - the probability that the sentence composed of words $(w_1, ..., w_T)$ in this order




This unsupervised learning problem can be framed as a sequence of supervised learning problems:


$p(W) = p(w_1) * p(w_2|w_1) * ... * p(w_T|w_1, ..., w_{T-1})$



If we have N sentences, each of them with T words / tokens, then we want to max:

$log p(W) = \sum_{n = 1}^N \sum_{i=1}^{T} log p(w_i | w_{<i})$




# **N-gram language model**

**Goal:** estimate the n-gram probabilities using counts of sequences of n consecutive words

Given a sequence of words $w$, we want to compute

  $P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1})$

Where $w_i$ is the i-th word of the sequence.

$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) = \frac{p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$

**Key Idea:** We can estimate the probabilities using counts of n-grams in our dataset 


# Model

# Loss Function and Optimizer

# Training and Testing

# Analysis & Examples